<a href="https://colab.research.google.com/github/driano1221/Desafio-de-Machine-Learning/blob/main/extrair_imgs_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import fitz
import re
import cv2
import shutil
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pdf_path = "/content/drive/MyDrive/Colab Notebooks/Cópia de PAT-RT-LAB-2612.21-001-Rev.03.pdf"
pasta_saida = "/content/drive/MyDrive/Colab Notebooks/saida"

In [ ]:
doc = fitz.open(pdf_path)

In [ ]:
id_pattern = re.compile(r"(?:Amostra|ID):\s*(.+)", re.IGNORECASE)

In [ ]:
# --- --------------FILTROS E CONFIGURACOES ---------------------------
TERMOS_PARA_IGNORAR = ["TESTEMUNHO", "TSTEMUNHO", "TESTEMUNHHO", "TESTEMNHO", "FERNANDO"]
LOGO_LARGURA = 117
LOGO_ALTURA = 128
LARGURA_MINIMA = 300
ALTURA_MINIMA = 150
TAMANHO_MINIMO_KB = 20
pagina_limite = 1083
texto = "Depois do Ensaio"
DPI = 300 # Resolução para renderizar a imagem. DPI mais alto = melhor qualidade.

In [ ]:
# Função auxiliar para unir retângulos (bounding boxes)

def unir_retangulos(rects):
    if not rects:
        return None
    final_rect = fitz.Rect(rects[0])
    for r in rects[1:]:
        final_rect.include_rect(r)
    return final_rect

In [ ]:
# Função para agrupar imagens próximas

def agrupar_imagens_proximas(page, tolerancia=5):
    infos = page.get_image_info(xrefs=True)
    if not infos:
        return []

    rects = [fitz.Rect(info['bbox']) for info in infos]
    grupos = []

    while rects:
        base_rect = rects.pop(0)
        grupo_atual = [base_rect]
        i = 0
        while i < len(rects):
            r = rects[i]
            # Se um retângulo está próximo do grupo atual, junte-o
            if unir_retangulos(grupo_atual).intersects(r.irect + (-tolerancia, -tolerancia, tolerancia, tolerancia)):
                grupo_atual.append(rects.pop(i))
                i = 0  # Reinicia a verificação pois o grupo mudou
            else:
                i += 1
        grupos.append(unir_retangulos(grupo_atual))
    return grupos


In [ ]:
os.makedirs(pasta_saida, exist_ok=True)

try:
    doc = fitz.open(pdf_path)
except Exception as e:
    print(f"Erro ao abrir o PDF: {e}")
    exit()

In [ ]:
dados_extraidos = []
contador_salvas = 0
contador_ignoradas = 0


In [ ]:
print(
    f"Iniciando extração com junção de imagens de: {os.path.basename(pdf_path)}")

for page_num in range(len(doc)):

    if page_num + 1 > pagina_limite:
        break

    page = doc.load_page(page_num)
    text = page.get_text()

    if texto.lower() in text.lower():

        id_match = id_pattern.search(text)

        if id_match:
            id_value = id_match.group(1).strip()

            ignorar_pagina = any(termo.upper() in id_value.upper()
                                for termo in TERMOS_PARA_IGNORAR)
            if ignorar_pagina:
                # print(f"  -> Página {page_num + 1} ignorada (contém termo proibido).")
                continue

            print(
                f"  --- ID '{id_value}' válido encontrado na página {page_num + 1}. Agrupando e renderizando imagens...")
            sanitized_id = re.sub(r'[\\/*?:"<>|\s]', '_', id_value)

            # --- NOVA LÓGICA ---
            # 1. Agrupa as imagens fatiadas
            areas_agrupadas = agrupar_imagens_proximas(page)

            # 2. Itera sobre as áreas agrupadas (imagens completas)
            for img_index, area_total in enumerate(areas_agrupadas, start=1):

                # 3. Renderiza a área completa como uma única imagem
                # Matriz de zoom para alta resolução
                mat = fitz.Matrix(DPI / 72, DPI / 72)
                pix = page.get_pixmap(clip=area_total, matrix=mat)

                # 4. Aplica os filtros na imagem JÁ RENDERIZADA E COMPLETA
                if (pix.width == LOGO_LARGURA and pix.height == LOGO_ALTURA):
                    contador_ignoradas += 1
                    continue

                if (pix.width < LARGURA_MINIMA or pix.height < ALTURA_MINIMA or len(pix.samples) < (TAMANHO_MINIMO_KB * 1024)):
                    contador_ignoradas += 1
                    continue

                # Se passou pelos filtros, salva
                # Salvar como PNG é mais seguro para pixmaps
                image_filename = f"pag_{page_num + 1:03}_ID_{sanitized_id}_img_{img_index}.png"
                output_path = os.path.join(pasta_saida, image_filename)
                pix.save(output_path)
                contador_salvas += 1

                dados_csv = {
                    'ID da Amostra': id_value,
                    'Pagina do PDF': page_num + 1,
                    'Nome do Arquivo': image_filename,
                    'Largura (px)': pix.width,
                    'Altura (px)': pix.height
                }

                dados_extraidos.append(dados_csv)

doc.close()

Iniciando extração com junção de imagens de: Cópia de PAT-RT-LAB-2612.21-001-Rev.03.pdf
  --- ID 'ITA-FJ00001 - AM002' válido encontrado na página 39. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM002' válido encontrado na página 46. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM002' válido encontrado na página 53. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM003' válido encontrado na página 59. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM003' válido encontrado na página 66. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM003' válido encontrado na página 73. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM004' válido encontrado na página 81. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM004' válido encontrado na página 89. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM006' válido encontrado na página 98. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ0000

In [ ]:
df = pd.DataFrame(dados_extraidos)
csv_path = os.path.join(pasta_saida, "relatorio_extracao.csv")
df.to_csv(csv_path, index=False, encoding='utf-8-sig')
print("-" * 50)
print("Extração concluída com sucesso!")
print(f"Total de imagens completas salvas: {contador_salvas}")
print(f"Total de imagens ignoradas (logos/pequenas): {contador_ignoradas}")
print(f"As imagens foram salvas na pasta: '{pasta_saida}'")

--------------------------------------------------
Extração concluída com sucesso!
Total de imagens completas salvas: 534
Total de imagens ignoradas (logos/pequenas): 139
As imagens foram salvas na pasta: '/content/drive/MyDrive/Colab Notebooks/saida'


In [ ]:
import fitz
import re
import cv2
import shutil
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

pdf_path = "/content/drive/MyDrive/Colab Notebooks/Cópia de PAT-RT-LAB-2612.21-001-Rev.03.pdf"
pasta_saida = "/content/drive/MyDrive/Colab Notebooks/vistas_separadas"

doc = fitz.open(pdf_path)

id_pattern = re.compile(r"(?:Amostra|ID):\s*(.+)", re.IGNORECASE)
registro_pattern = re.compile(r"Registro:\s*(\d+)", re.IGNORECASE)  # Novo padrão para capturar o registro

# --- --------------FILTROS E CONFIGURACOES ---------------------------
TERMOS_PARA_IGNORAR = ["TESTEMUNHO", "TSTEMUNHO", "TESTEMUNHHO", "TESTEMNHO", "FERNANDO"]
LOGO_LARGURA = 117
LOGO_ALTURA = 128
LARGURA_MINIMA = 300
ALTURA_MINIMA = 150
TAMANHO_MINIMO_KB = 20
pagina_limite = 1083
texto = "Depois do Ensaio"
DPI = 300 # Resolução para renderizar a imagem. DPI mais alto = melhor qualidade.

# Função auxiliar para unir retângulos (bounding boxes)

def unir_retangulos(rects):
    if not rects:
        return None
    final_rect = fitz.Rect(rects[0])
    for r in rects[1:]:
        final_rect.include_rect(r)
    return final_rect

# Função para agrupar imagens próximas

def agrupar_imagens_proximas(page, tolerancia=5):
    infos = page.get_image_info(xrefs=True)
    if not infos:
        return []

    rects = [fitz.Rect(info['bbox']) for info in infos]
    grupos = []

    while rects:
        base_rect = rects.pop(0)
        grupo_atual = [base_rect]
        i = 0
        while i < len(rects):
            r = rects[i]
            # Se um retângulo está próximo do grupo atual, junte-o
            if unir_retangulos(grupo_atual).intersects(r.irect + (-tolerancia, -tolerancia, tolerancia, tolerancia)):
                grupo_atual.append(rects.pop(i))
                i = 0  # Reinicia a verificação pois o grupo mudou
            else:
                i += 1
        grupos.append(unir_retangulos(grupo_atual))
    return grupos

os.makedirs(pasta_saida, exist_ok=True)

try:
    doc = fitz.open(pdf_path)
except Exception as e:
    print(f"Erro ao abrir o PDF: {e}")
    exit()

dados_extraidos = []
contador_salvas = 0
contador_ignoradas = 0

print(
    f"Iniciando extração com junção de imagens de: {os.path.basename(pdf_path)}")

for page_num in range(len(doc)):

    if page_num + 1 > pagina_limite:
        break

    page = doc.load_page(page_num)
    text = page.get_text()

    if texto.lower() in text.lower():

        id_match = id_pattern.search(text)
        registro_match = registro_pattern.search(text)  # Busca pelo registro

        if id_match and registro_match:  # Agora precisa ter tanto ID quanto Registro
            id_value = id_match.group(1).strip()
            registro_value = registro_match.group(1).strip()  # Captura o número do registro

            ignorar_pagina = any(termo.upper() in id_value.upper()
                                for termo in TERMOS_PARA_IGNORAR)
            if ignorar_pagina:
                # print(f"  -> Página {page_num + 1} ignorada (contém termo proibido).")
                continue

            print(
                f"  --- ID '{id_value}' e Registro '{registro_value}' encontrados na página {page_num + 1}. Agrupando e renderizando imagens...")

            # --- NOVA LÓGICA ---
            # 1. Agrupa as imagens fatiadas
            areas_agrupadas = agrupar_imagens_proximas(page)

            # 2. Itera sobre as áreas agrupadas (imagens completas)
            for img_index, area_total in enumerate(areas_agrupadas, start=1):

                # 3. Renderiza a área completa como uma única imagem
                # Matriz de zoom para alta resolução
                mat = fitz.Matrix(DPI / 72, DPI / 72)
                pix = page.get_pixmap(clip=area_total, matrix=mat)

                # 4. Aplica os filtros na imagem JÁ RENDERIZADA E COMPLETA
                if (pix.width == LOGO_LARGURA and pix.height == LOGO_ALTURA):
                    contador_ignoradas += 1
                    continue

                if (pix.width < LARGURA_MINIMA or pix.height < ALTURA_MINIMA or len(pix.samples) < (TAMANHO_MINIMO_KB * 1024)):
                    contador_ignoradas += 1
                    continue

                # Se passou pelos filtros, salva com o novo formato de nome
                image_filename = f"{registro_value}_vista{img_index}.jpg"  # Novo formato
                output_path = os.path.join(pasta_saida, image_filename)

                # Salvar como JPG (convertendo do PNG)
                pix.save(output_path)
                contador_salvas += 1

                dados_csv = {
                    'ID da Amostra': id_value,
                    'Registro': registro_value,  # Adiciona o registro no CSV
                    'Pagina do PDF': page_num + 1,
                    'Nome do Arquivo': image_filename,
                    'Largura (px)': pix.width,
                    'Altura (px)': pix.height
                }

                dados_extraidos.append(dados_csv)

doc.close()

df = pd.DataFrame(dados_extraidos)
csv_path = os.path.join(pasta_saida, "relatorio_extracao.csv")
df.to_csv(csv_path, index=False, encoding='utf-8-sig')
print("-" * 50)
print("Extração concluída com sucesso!")
print(f"Total de imagens completas salvas: {contador_salvas}")
print(f"Total de imagens ignoradas (logos/pequenas): {contador_ignoradas}")
print(f"As imagens foram salvas na pasta: '{pasta_saida}'")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Iniciando extração com junção de imagens de: Cópia de PAT-RT-LAB-2612.21-001-Rev.03.pdf
  --- ID 'ITA-FJ00001 - AM002' e Registro '567' encontrados na página 39. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM002' e Registro '568' encontrados na página 46. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM002' e Registro '569' encontrados na página 53. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM003' e Registro '570' encontrados na página 59. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM003' e Registro '571' encontrados na página 66. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM003' e Registro '572' encontrados na página 73. Agrupando e renderizando imagens...
  --- ID 'ITA-FJ00001 - AM004' e Registro '573' encontrados na página 81. Agrupando e renderizando imagens...
  --- ID 'ITA

In [ ]:
import fitz
import re
import os
import numpy as np
import cv2
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')

# Configurações específicas para a página 682 (registro 719)
pdf_path = "/content/drive/MyDrive/Colab Notebooks/Cópia de PAT-RT-LAB-2612.21-001-Rev.03.pdf"
pasta_saida = "/content/drive/MyDrive/Colab Notebooks/vistas_separadas_pagina_719"
pagina_alvo = 682  # Índice 681 corresponde à página 682

# Criar pasta de saída
os.makedirs(pasta_saida, exist_ok=True)

# Configurações técnicas
DPI = 300
MARGEM_LATERAL = 0.05  # 5% da largura da página como margem
NUM_VISTAS = 4  # Número de vistas esperadas

def crop_image(img):
    """Recorta as margens brancas de uma imagem."""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return img

    # Encontrar o maior contorno (a área principal)
    max_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(max_contour)

    # Adicionar pequena margem ao redor
    padding = 5
    x = max(0, x - padding)
    y = max(0, y - padding)
    w = min(img.shape[1] - x, w + 2*padding)
    h = min(img.shape[0] - y, h + 2*padding)

    return img[y:y+h, x:x+w]

print(f"Iniciando processamento ESPECÍFICO para a página 682 (índice {pagina_alvo})")

try:
    doc = fitz.open(pdf_path)
    print(f"PDF aberto com sucesso. Total de páginas: {len(doc)}")

    page = doc.load_page(pagina_alvo)
    print(f"Página {pagina_alvo + 1} carregada com sucesso")

    # Extrair texto para identificar o registro
    text = page.get_text()
    print(f"Texto extraído (primeiros 200 chars): {text[:200]}...")

    registro_pattern = re.compile(r"Registro:\s*(\d+)", re.IGNORECASE)
    registro_match = registro_pattern.search(text)

    if registro_match:
        registro_value = registro_match.group(1).strip()
        print(f"Registro identificado: {registro_value}")

        # Renderizar a página inteira em alta resolução
        mat = fitz.Matrix(DPI / 72, DPI / 72)
        pix = page.get_pixmap(matrix=mat)
        img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
        if pix.n == 4:
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        # Definir áreas de corte baseadas no layout padrão
        height, width = img.shape[:2]
        margin = int(width * MARGEM_LATERAL)
        vista_width = (width - 2 * margin) // NUM_VISTAS

        # Coordenadas finais para cada vista
        areas = [
            (margin, 0, vista_width, height),
            (margin + vista_width, 0, vista_width, height),
            (margin + 2 * vista_width, 0, vista_width, height),
            (margin + 3 * vista_width, 0, vista_width, height)
        ]

        for i, (x, y, w, h) in enumerate(areas, start=1):
            # Recortar a área
            vista = img[y:y+h, x:x+w]

            # Remover margens brancas
            vista_crop = crop_image(vista)

            # Salvar a vista
            image_filename = f"{registro_value}_vista{i}.jpg"
            output_path = os.path.join(pasta_saida, image_filename)
            cv2.imwrite(output_path, vista_crop)
            print(f"Vista {i} salva em: {output_path}")

    else:
        print("Registro não encontrado. Usando nome padrão...")
        registro_value = "719"
        # Processo similar usando nome padrão

    doc.close()
    print("\nProcessamento concluído!")
    print(f"Imagens salvas em: {pasta_saida}")

except Exception as e:
    print(f"Erro: {str(e)}")
    import traceback
    traceback.print_exc()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Iniciando processamento ESPECÍFICO para a página 682 (índice 682)
PDF aberto com sucesso. Total de páginas: 1531
Página 683 carregada com sucesso
Texto extraído (primeiros 200 chars): Cliente:
Vale S.A.
ID:
ITA-FJ00012 - AM0017
Registro:
719
Depois do Ensaio
PATROL INVESTIGAÇÕES GEOTECNICAS LTDA.
RELATÓRIO FOTOGRÁFICO
...
Registro identificado: 719
Vista 1 salva em: /content/drive/MyDrive/Colab Notebooks/vistas_separadas_pagina_719/719_vista1.jpg
Vista 2 salva em: /content/drive/MyDrive/Colab Notebooks/vistas_separadas_pagina_719/719_vista2.jpg
Vista 3 salva em: /content/drive/MyDrive/Colab Notebooks/vistas_separadas_pagina_719/719_vista3.jpg
Vista 4 salva em: /content/drive/MyDrive/Colab Notebooks/vistas_separadas_pagina_719/719_vista4.jpg

Processamento concluído!
Imagens salvas em: /content/drive/MyDrive/Colab Notebooks/vistas_separadas_pagina_719


In [ ]:
import pandas as pd
import os
from collections import Counter

# Caminho do arquivo CSV gerado
csv_path = "/content/drive/MyDrive/Colab Notebooks/vistas_separadas/relatorio_extracao.csv"
pasta_imagens = "/content/drive/MyDrive/Colab Notebooks/vistas_separadas"

# Verifica se o arquivo CSV existe
if not os.path.exists(csv_path):
    print("Arquivo CSV não encontrado. Execute primeiro a extração das imagens.")
else:
    # Lê o CSV
    df = pd.read_csv(csv_path)

    # Extrai o número da vista do nome do arquivo
    df['numero_vista'] = df['Nome do Arquivo'].str.extract(r'vista(\d+)\.jpg').astype(int)

    # Conta vistas por registro
    vistas_por_registro = df.groupby('Registro').agg({
        'numero_vista': ['min', 'max', 'count'],
        'ID da Amostra': 'first'
    }).reset_index()

    # Renomeia colunas
    vistas_por_registro.columns = ['Registro', 'Vista_Min', 'Vista_Max', 'Total_Vistas', 'ID_Amostra']

    print("📊 ESTATÍSTICAS DE VISTAS POR AMOSTRA")
    print("=" * 60)

    # Mostra estatísticas gerais
    total_amostras = len(vistas_por_registro)
    total_vistas = df.shape[0]

    print(f"Total de amostras processadas: {total_amostras}")
    print(f"Total de vistas extraídas: {total_vistas}")
    print()

    # Análise de distribuição de vistas
    distribuicao = vistas_por_registro['Total_Vistas'].value_counts().sort_index()
    print("📈 Distribuição de amostras por número de vistas:")
    for num_vistas, count in distribuicao.items():
        print(f"  {num_vistas} vistas: {count} amostras")
    print()

    # Verifica amostras com número incomum de vistas
    media_vistas = vistas_por_registro['Total_Vistas'].mean()
    print(f"📊 Média de vistas por amostra: {media_vistas:.2f}")
    print()

    # Mostra amostras com menor e maior número de vistas
    min_vistas = vistas_por_registro['Total_Vistas'].min()
    max_vistas = vistas_por_registro['Total_Vistas'].max()

    amostras_menos_vistas = vistas_por_registro[vistas_por_registro['Total_Vistas'] == min_vistas]
    amostras_mais_vistas = vistas_por_registro[vistas_por_registro['Total_Vistas'] == max_vistas]

    print(f"🔽 Amostras com menor número de vistas ({min_vistas}):")
    for _, row in amostras_menos_vistas.iterrows():
        print(f"  Registro: {row['Registro']} | ID: {row['ID_Amostra']} | Vistas: {row['Total_Vistas']}")
    print()

    print(f"🔼 Amostras com maior número de vistas ({max_vistas}):")
    for _, row in amostras_mais_vistas.iterrows():
        print(f"  Registro: {row['Registro']} | ID: {row['ID_Amostra']} | Vistas: {row['Total_Vistas']}")
    print()

    # Detalhamento por tipo de amostra (ID)
    print("📋 ANÁLISE POR TIPO DE AMOSTRA (ID)")
    print("=" * 60)

    vistas_por_id = df.groupby('ID da Amostra').agg({
        'Registro': 'nunique',
        'Nome do Arquivo': 'count'
    }).reset_index()
    vistas_por_id.columns = ['ID_Amostra', 'Qtd_Registros', 'Total_Vistas']

    # Calcula média de vistas por registro para cada ID
    media_por_id = df.groupby('ID da Amostra')['numero_vista'].agg(['min', 'max', 'mean', 'count']).reset_index()
    media_por_id.columns = ['ID_Amostra', 'Vista_Min', 'Vista_Max', 'Media_Vistas', 'Total_Vistas']

    print("Estatísticas por tipo de amostra:")
    for _, row in media_por_id.iterrows():
        print(f"  ID: {row['ID_Amostra']}")
        print(f"    Total de vistas: {row['Total_Vistas']}")
        print(f"    Faixa de vistas: {row['Vista_Min']} a {row['Vista_Max']}")
        print(f"    Média: {row['Media_Vistas']:.2f}")
        print()

    # Verifica se todas as amostras têm o mesmo número de vistas
    todos_iguais = len(distribuicao) == 1
    if todos_iguais:
        print("✅ Todas as amostras têm o mesmo número de vistas!")
    else:
        print("⚠️  ATENÇÃO: Nem todas as amostras têm o mesmo número de vistas!")
        print("   Verifique as amostras listadas acima.")

    print("\n" + "=" * 60)
    print("🔍 DETALHAMENTO COMPLETO POR REGISTRO")
    print("=" * 60)

    # Ordena por número de vistas (decrescente) para facilitar identificação de problemas
    vistas_por_registro = vistas_por_registro.sort_values('Registro', ascending=False)

    for _, row in vistas_por_registro.iterrows():
        print(f"Registro: {row['Registro']} | ID: {row['ID_Amostra']} | Vistas: {row['Total_Vistas']} "
              f"(de {row['Vista_Min']} a {row['Vista_Max']})")

📊 ESTATÍSTICAS DE VISTAS POR AMOSTRA
Total de amostras processadas: 139
Total de vistas extraídas: 534

📈 Distribuição de amostras por número de vistas:
  1 vistas: 2 amostras
  2 vistas: 4 amostras
  3 vistas: 8 amostras
  4 vistas: 125 amostras

📊 Média de vistas por amostra: 3.84

🔽 Amostras com menor número de vistas (1):
  Registro: 589 | ID: ITA-FJ00001 - AM010 | Vistas: 1
  Registro: 784 | ID: ITA-FJ00003 - AM0002 | Vistas: 1

🔼 Amostras com maior número de vistas (4):
  Registro: 567 | ID: ITA-FJ00001 - AM002 | Vistas: 4
  Registro: 568 | ID: ITA-FJ00001 - AM002 | Vistas: 4
  Registro: 569 | ID: ITA-FJ00001 - AM002 | Vistas: 4
  Registro: 570 | ID: ITA-FJ00001 - AM003 | Vistas: 4
  Registro: 571 | ID: ITA-FJ00001 - AM003 | Vistas: 4
  Registro: 572 | ID: ITA-FJ00001 - AM003 | Vistas: 4
  Registro: 573 | ID: ITA-FJ00001 - AM004 | Vistas: 4
  Registro: 574 | ID: ITA-FJ00001 - AM004 | Vistas: 4
  Registro: 578 | ID: ITA-FJ00001 - AM006 | Vistas: 4
  Registro: 579 | ID: ITA-FJ00001

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 79.3 MB/s eta 0:00:00


In [ ]:
import os
import re
from collections import defaultdict

# Caminho da pasta
pasta = "/content/drive/MyDrive/Colab Notebooks/vistas_separadas"

# Dicionário para contar vistas por registro
registros = defaultdict(int)

# Listar arquivos e contar vistas
for arquivo in os.listdir(pasta):
    if arquivo.endswith('.jpg'):
        # Extrair número do registro (formato: 123_vista1.jpg)
        match = re.match(r'(\d+)_vista\d+\.jpg', arquivo)
        if match:
            registro = match.group(1)
            registros[registro] += 1

# Filtrar registros que não têm 4 vistas
print("📋 REGISTROS COM NÚMERO IRREGULAR DE VISTAS:")
print("=" * 50)

irregulares = [(reg, count) for reg, count in registros.items() if count != 4]

if irregulares:
    for registro, vistas in sorted(irregulares, key=lambda x: int(x[0])):
        print(f"Registro {registro}: {vistas} vistas")
    print(f"\nTotal de registros irregulares: {len(irregulares)}")
else:
    print("✅ Todos os registros têm 4 vistas!")

print(f"Total geral de registros: {len(registros)}")

📋 REGISTROS COM NÚMERO IRREGULAR DE VISTAS:
Registro 589: 1 vistas
Registro 592: 3 vistas
Registro 610: 3 vistas
Registro 618: 2 vistas
Registro 661: 2 vistas
Registro 676: 3 vistas
Registro 678: 3 vistas
Registro 682: 3 vistas
Registro 708: 3 vistas
Registro 719: 2 vistas
Registro 746: 2 vistas
Registro 776: 2 vistas
Registro 784: 1 vistas
Registro 832: 3 vistas

Total de registros irregulares: 14
Total geral de registros: 139
